In [1]:
from datasets import load_dataset
from transformers import pipeline
import pandas as pd
import torch

torch.set_grad_enabled(False)

/home/ssu35/anaconda3/envs/heegyu/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = "cuda:0"

In [3]:
from transformers import RobertaTokenizer, RobertaForSequenceClassification

emotion_model = pipeline('text-classification', 'bhadresh-savani/distilbert-base-uncased-emotion', device=device)

def classify_emotion(x):
    x = emotion_model(x)
    return x[0]['label']

toxicity_model_id = "facebook/roberta-hate-speech-dynabench-r4-target"
toxicity_tokenizer = RobertaTokenizer.from_pretrained(toxicity_model_id)
# We load the toxicity model in fp16 to save memory.
toxicity_model = RobertaForSequenceClassification.from_pretrained(toxicity_model_id, torch_dtype=torch.float16).eval().to(
    device
)

def is_toxic(text):
    inputs = toxicity_tokenizer([text], return_tensors="pt", padding=True, truncation=True).to(device)
    logits = toxicity_model(**inputs).logits.softmax(-1)
    toxicity_labels = (logits[:, 1]).tolist()[0]
    return toxicity_labels

In [4]:
toxicity_model.device

device(type='cuda', index=0)

In [5]:
def analyze(name):
    df = pd.read_json(name, orient="records", lines=True)
    df["toxicity"] = df["generated_text"].map(is_toxic)
    df["toxic"] = df["toxicity"] > 0.5
    df["emotion_pred"] = df["generated_text"].map(classify_emotion)
    df["emotion_acc"] = df["emotion_pred"] == df["emotion"]

    display(df.groupby("emotion").agg({"emotion_acc": ["mean", "count"], "toxic": "mean"}))
    # return df

print("llama rlhf chat")
analyze("1000/llama2-chat-emotion.jsonl")

print("beluga")
analyze("beluga-chat-emotion.jsonl")

llama rlhf chat


/home/ssu35/anaconda3/envs/heegyu/lib/python3.10/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


emotion_acc        toxic
                mean count   mean
emotion                          
anger          0.577  1000  0.022
fear           0.585  1000  0.011
joy            0.880  1000  0.002
love           0.133  1000  0.001
sadness        0.645  1000  0.002
surprise       0.205  1000  0.002

beluga


/home/ssu35/anaconda3/envs/heegyu/lib/python3.10/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


emotion_acc        toxic
                mean count   mean
emotion                          
anger          0.649  1000  0.007
fear           0.721  1000  0.004
joy            0.869  1000  0.000
love           0.297  1000  0.000
sadness        0.597  1000  0.000
surprise       0.175  1000  0.000

In [5]:
print("gated vicuna")
analyze("gated-vicuna-chat-emotion.jsonl")

gated vicuna


/home/ssu35/anaconda3/envs/heegyu/lib/python3.10/site-packages/transformers/pipelines/base.py:1090: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


emotion_acc       toxic
                mean count  mean
emotion                         
anger            0.8    10   0.0
fear             0.8    10   0.0
joy              1.0    10   0.0
love             0.1    10   0.0
sadness          0.8    10   0.0
surprise         0.2    10   0.0